In [1]:
from carla import Benchmark
from IPython.display import display
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
scm = CausalModel("credit")
dataset = scm.generate_dataset(1000)

display(dataset.df)

,label,x1,x2,x3,x4,x6,x5,x7
0,1.0,1.0,1.389002,0.004628,0.970468,2.988175,4.432239,4.267007
1,0.0,1.0,-7.931253,-0.192720,-1.307835,-0.822434,-2.697714,-4.419209
2,0.0,0.0,4.125594,-0.264288,-3.900621,-1.893516,0.129825,5.347141
3,1.0,0.0,-8.425271,-0.098301,1.776086,-0.204366,1.432725,7.357849
4,1.0,1.0,-0.742683,-0.146454,2.567742,2.540867,3.782052,-4.821194
...,...,...,...,...,...,...,...,...
995,0.0,0.0,-13.542408,-0.367479,-7.400572,-0.354547,-11.466550,-13.037549
996,1.0,1.0,9.414795,0.054111,3.918685,2.589903,5.199423,3.065171
997,1.0,0.0,0.440604,0.141155,1.925721,1.300247,6.983869,4.849467
998,0.0,0.0,-1.348053,-0.200820,3.460917,-5.199851,1.310833,-7.935876


In [3]:
#Model 

training_params = {"lr": 0.01, "epochs": 3, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.49733333333333335, balance on test set 0.484
Epoch 0/2
----------
train Loss: 0.4544 Acc: 0.8120

test Loss: 0.4497 Acc: 0.8360

Epoch 1/2
----------
train Loss: 0.4099 Acc: 0.8360

test Loss: 0.4098 Acc: 0.8400

Epoch 2/2
----------
train Loss: 0.3980 Acc: 0.8373

test Loss: 0.4183 Acc: 0.8360



In [4]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,label,x1,x2,x3,x4,x6,x5,x7
1,0.0,1.0,-7.931253,-0.192720,-1.307835,-0.822434,-2.697714,-4.419209
10,0.0,0.0,-10.904226,-0.243402,-1.922213,-0.661480,-3.415021,4.228990
11,0.0,0.0,-9.683098,-0.331825,-4.840960,-0.905047,-6.527149,-7.360349
12,0.0,1.0,-4.060140,0.104382,0.540323,0.978483,3.491786,-6.035425
14,1.0,1.0,-7.198979,-0.223423,2.690985,0.637859,4.926449,-1.489003
16,0.0,0.0,-10.611073,0.030899,-2.706385,-2.530421,-3.489129,3.995988
17,0.0,0.0,4.823524,-0.084034,-0.400280,-0.282182,0.775708,-8.586262
18,0.0,0.0,-5.319518,-0.157119,-2.348676,-1.297170,-5.737461,0.925753
19,0.0,1.0,-13.637587,-0.028605,1.035043,1.061113,2.838433,-4.789891
21,0.0,0.0,2.530850,-0.122564,-3.451096,0.619606,-3.339138,-2.227577


In [5]:
hyperparams = {"loss_type": "MSE", "binary_cat_features": False}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
cfs = recourse_method.get_counterfactuals(test_factual)
#from carla.models.negative_instances import predict_negative_instances
#from carla.recourse_methods.catalog.causal_recourse import (
#    CausalRecourse,
#    constraints,
#    samplers,
#)

#hyperparams = {
#    "optimization_approach": "brute_force",
#    "num_samples": 10,
#    "scm": scm,
#    "constraint_handle": constraints.point_constraint,
#    "sampler_handle": samplers.sample_true_m0,
#}
#cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

AttributeError: 'NoneType' object has no attribute 'get_feature_names'

In [ ]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    #TODO change everything hard coded
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    labels=endogenous_variables['x9'][0]
    return labels

In [ ]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
    def get_evaluation(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactual.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactual,cf_label, mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

class path_checker():
    pass


metric = Sematic(ml_model,scm)
result=metric.get_evaluation(np.array([]),df_cfs.iloc[0],mapping_dict)
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


NameError: name 'df_cfs' is not defined